---
title: "Lab 3: Platforms & Network Effects"
subtitle: "Adoption simulation and sensitivity"
format:
  html:
    toc: false
    number-sections: true
execute:
  echo: true
  warning: false
  message: false
---

::: callout-note
### Expected Time
- FIN510: Seminar hands‑on ≈ 60 min; 
- Directed learning extensions ≈ 90–120 min
- FIN720: Computer lab ≈ 120 min
:::

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/quinfer/fin510-colab-notebooks/blob/main/labs/lab03_platforms.ipynb)

## Objectives

- Implement the two‑sided adoption simulation
- Explore pricing and congestion sensitivities

## Simulation Function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_platform(T=60, a_u=0.02, a_m=0.015, beta_um=0.6, beta_mu=0.5,
                      price_u=0.0, price_m=0.03, gamma_u=0.0, gamma_m=0.0):
    Nu = np.zeros(T+1); Nm = np.zeros(T+1)
    Nu[0]=0.02; Nm[0]=0.02
    for t in range(T):
        du = a_u*(1-Nu[t])*max(0.0, (1 + beta_um*Nm[t] - price_u)) - gamma_u*Nu[t]**2
        dm = a_m*(1-Nm[t])*max(0.0, (1 + beta_mu*Nu[t] - price_m)) - gamma_m*Nm[t]**2
        Nu[t+1] = np.clip(Nu[t]+du, 0, 1)
        Nm[t+1] = np.clip(Nm[t]+dm, 0, 1)
    return Nu, Nm

## Task 1 — Scenario Comparison

In [ ]:
scenarios = {
    'Merchant‑pays': dict(price_u=0.0, price_m=0.03),
    'User‑pays':     dict(price_u=0.03, price_m=0.0),
    'Both free':     dict(price_u=0.0, price_m=0.0),
    'Congestion':    dict(price_u=0.0, price_m=0.03, gamma_u=0.15, gamma_m=0.10),
}

plt.figure(figsize=(10,6))
for label, params in scenarios.items():
    Nu, Nm = simulate_platform(**params)
    plt.plot(Nu, label=f'U — {label}')
    plt.plot(Nm, linestyle='--', label=f'M — {label}')
plt.title('Two‑Sided Adoption Scenarios'); plt.xlabel('Time'); plt.ylabel('Participation (0..1)')
plt.grid(alpha=0.3); plt.legend(ncol=2, fontsize=8); plt.tight_layout()
Nu, Nm = simulate_platform()
assert Nu.min() >= 0 and Nu.max() <= 1 and Nm.min() >= 0 and Nm.max() <= 1
print("Scenario simulation checks passed ✔")

## Task 2 — Price Sensitivity Heatmap

In [ ]:
grid = np.linspace(0.0, 0.06, 13)
Z = np.zeros((len(grid), len(grid)))
for i, pu in enumerate(grid):
    for j, pm in enumerate(grid):
        Nu, Nm = simulate_platform(price_u=pu, price_m=pm)
        Z[i,j] = Nu[-1]

plt.figure(figsize=(7,5))
im = plt.imshow(Z, origin='lower', extent=[grid.min(), grid.max(), grid.min(), grid.max()],
                aspect='auto', cmap='viridis')
plt.colorbar(im, label='Final user participation')
plt.xlabel('price_m'); plt.ylabel('price_u'); plt.title('User Participation vs Prices')
plt.tight_layout()
assert Z.min() >= 0 and Z.max() <= 1
print("Heatmap bounds checks passed ✔")

Deliverable: 200–300 words on subsidy logic and congestion governance for one scenario.

::: callout-tip
### Troubleshooting
- Flat lines: check parameters and ensure time horizon `T` is long enough.
- Values >1 or <0: verify clipping and parameter signs.
:::

::: callout-note
### Further Reading (Hilpisch 2019)
- See: [Hilpisch Code Resources](../resources/hilpisch-code.qmd) — Week 0/2
- While the platform simulation is domain‑specific, Chapter 13 diagnostics (learning curves, residuals) provide squared‑loss tools you can apply when regressing simulated outcomes on parameters.
:::